<a href="https://colab.research.google.com/github/saks0106/DeepLearning_NLP/blob/main/10_Neural_Machine_Translation_using_HuggingFace_%F0%9F%A4%97_Transformers_by_Neuralearn_ai_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.5 MB/s eta 0:00:00


In [2]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 6.4 MB/s eta 0:00:00


# Imports

In [3]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import re
import string
import time
import evaluate
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import (DataCollatorWithPadding,create_optimizer,AutoTokenizer,DataCollatorForSeq2Seq,
                          T5TokenizerFast,T5ForConditionalGeneration,TFAutoModelForSeq2SeqLM)

In [4]:
BATCH_SIZE=64

# Data Preparation for Bert Model

In [5]:
!wget https://www.manythings.org/anki/fra-eng.zip

--2023-07-04 17:22:30--  https://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7420323 (7.1M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.08M  18.9MB/s    in 0.4s    

2023-07-04 17:22:30 (18.9 MB/s) - ‘fra-eng.zip’ saved [7420323/7420323]



In [6]:
!unzip "/content/fra-eng.zip" -d "/content/dataset/"

Archive:  /content/fra-eng.zip
  inflating: /content/dataset/_about.txt  
  inflating: /content/dataset/fra.txt  


In [7]:
dataset=load_dataset('text', data_files='/content/dataset/fra.txt')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-2e9867a9d5f7f45d/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 217975
    })
})

In [9]:
dataset['train'][100000]['text'].split('\t')[:-1]

["There's a leak in the roof.", 'Il y a une fuite dans la toiture.']

In [10]:
model_id="t5-small"
tokenizer=T5TokenizerFast.from_pretrained(model_id)

In [11]:
prefix = "translate English to French: "

In [12]:
def preprocess_function(examples):

  inputs = [prefix + example.split('\t')[0] for example in examples['text']]
  targets = [example.split('\t')[1] for example in examples['text']]

  model_inputs = tokenizer(inputs, text_target=targets,max_length=128, truncation=True)
  return model_inputs

In [13]:
tokenized_dataset=dataset.map(preprocess_function,batched=True)

Map:   0%|          | 0/217975 [00:00<?, ? examples/s]

In [14]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 217975
    })
})

In [15]:
tokenized_dataset['train'][1000]

{'text': 'Grab this.\tAttrapez ça !\tCC-BY 2.0 (France) Attribution: tatoeba.org #1972637 (CK) & #1973298 (sacredceltic)',
 'input_ids': [13959, 1566, 12, 2379, 10, 15004, 48, 5, 1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [486, 16081, 457, 3664, 3, 55, 1]}

In [16]:
cleaned_dataset = tokenized_dataset.remove_columns("text")

In [17]:
cleaned_dataset['train'][1000]

{'input_ids': [13959, 1566, 12, 2379, 10, 15004, 48, 5, 1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [486, 16081, 457, 3664, 3, 55, 1]}

In [18]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,model=model, return_tensors="tf")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
tf_train_set = cleaned_dataset["train"].to_tf_dataset(
    shuffle=True,
    batch_size=BATCH_SIZE,
    collate_fn=data_collator,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
train_data=tf_train_set.take(int(0.9*len(tf_train_set)))
val_data=tf_train_set.skip(int(0.9*len(tf_train_set)))

In [ ]:
for i in val_data.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(64, 27), dtype=int64, numpy=
array([[13959,  1566,    12, ...,     0,     0,     0],
       [13959,  1566,    12, ...,     0,     0,     0],
       [13959,  1566,    12, ...,     0,     0,     0],
       ...,
       [13959,  1566,    12, ...,     0,     0,     0],
       [13959,  1566,    12, ...,     0,     0,     0],
       [13959,  1566,    12, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(64, 27), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>, 'labels': <tf.Tensor: shape=(64, 30), dtype=int64, numpy=
array([[  180, 17560,  9716, ...,  -100,  -100,  -100],
       [ 2740,     3,    29, ...,     3,    58,     1],
       [    3, 14756,  9472, ...,  -100,  -100,  -100],
       ...,
       [  276,   154,  8831, ...,  -100,  -100,  -100],
       [ 3006,   259

# Modeling

In [ ]:
model.summary()

Model: "tft5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (TFSharedEmbeddings)  multiple                 16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  18881280  
                                                                 
 decoder (TFT5MainLayer)     multiple                  25175808  
                                                                 
Total params: 60,506,624
Trainable params: 60,506,624
Non-trainable params: 0
_________________________________________________________________


# Training

In [ ]:
num_epochs = 3
num_train_steps = len(train_data) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
model.fit(
  train_data,
  validation_data=val_data,
  epochs=3
)

Epoch 1/3
2777/2777 [==============================] - 789s 276ms/step - loss: 0.8623 - val_loss: 0.6455
Epoch 2/3
2685/2777 [============================>.] - ETA: 24s - loss: 0.7890

KeyboardInterrupt: ignored

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Evaluation

In [ ]:
metric = evaluate.load("sacrebleu")

In [ ]:
all_preds = []
all_labels = []

for batch in val_data:
  predictions = model.generate(
      input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
  )
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = batch["labels"].numpy()
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  all_preds.extend(decoded_preds)
  all_labels.extend(decoded_labels)

result = metric.compute(predictions=all_preds, references=all_labels)
print(result)

/usr/local/lib/python3.7/dist-packages/transformers/generation_tf_utils.py:1602: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


{'score': 35.967454646883255, 'counts': [3048, 1714, 1047, 634], 'totals': [4775, 4135, 3495, 2858], 'precisions': [63.832460732984295, 41.45102781136639, 29.957081545064376, 22.18334499650105], 'bp': 0.987720001251615, 'sys_len': 4775, 'ref_len': 4834}


In [ ]:
decoded_preds

['Nous devons savoir qui il est et pour qui il travaille.',
 "Vous avez toutes l'intention de l'utiliser.",
 "Je ne peux pas m'en rendre compte.",
 'Merci pour votre temps.',
 "Jour après jour, le chien s'est assis à l'",
 "J'ai eu un bon temps hier.",
 'Je veux me mettre en forme, et je travaille chaque jour.',
 'Elle a été enthousiaste dans son travail.',
 'Combien de temps avez-vous séjourné?',
 "Vous n'avez jamais eu d'ami, avez-vous",
 " l'intérieur du corps, il y a une fente",
 "L'air est pour nous ce qu'il faut à poissonr.",
 'Les taux de divorce pourraient atteindre bientôt un plateau.',
 'Nous avons été des partenaires.',
 "Ouvrir les fenêtres pour laisser l'air dans une certaine mesure.",
 'Je ne vous ai jamais vu comme ça auparavant.',
 "C'est un glissement.",
 "Quelqu'un devrait en aviser le prochain parent.",
 "Tout s'est déroulé sans heurt.",
 'Il a déjà été pris en charge.',
 "C'est un bon journal, n'est-ce pas?",
 "Elle l'a vu continuer de se battre aussi dur qu'il pouv

In [ ]:
decoded_labels

['Nous devons découvrir qui il est et pour qui il travaille.',
 'Celui-ci est tout à toi.',
 'Je déteste voler.',
 'Je vous remercie pour votre temps.',
 'Jour après jour, le chien était assis en attendant son maître face à la station.',
 'Je me suis bien amusé hier.',
 "Je veux me mettre en forme alors je m'entraîne tous les jours.",
 'Elle était absorbée par son travail.',
 'Combien de temps êtes-vous resté?',
 "Tu n'as jamais eu de petit copain, n'est-ce pas?",
 'Brûle le corps.',
 "L'air est à l'homme ce que l'eau est aux poissons.",
 'Les taux de divorce pourraient bientôt atteindre un plateau.',
 'Nous étions partenaires.',
 "Ouvrez les fenêtres pour laisser rentrer un peu d'air.",
 'Je ne vous avais jamais vu ainsi auparavant.',
 "C'en est une scabreuse.",
 "Quelqu'un devrait informer les proches parents.",
 "Tout s'est déroulé en douceur.",
 "On s'en est déjà occupé.",
 "C'est un bon journal, n'est-ce pas?",
 "Elle le regarda continuer à se battre aussi fort qu'il pouvait.",
 "

# Testing


In [ ]:
input_text="Have you ever played soccer under the rain, with your friends? "
input_text="Google Translate is a multilingual neural machine translation service developed by Google to translate text, documents and websites from one language into another."
input_text=prefix + input_text
tokenized=tokenizer([input_text], return_tensors='tf')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[    0  1163 30355    15   259    73   313    20 16486  1575    20  1437
  24228    15  1249   697    76    15 23609   260  1163   171 25639    60
    146 10062     6    93  2691     3    15    17    93  1471  1620     3
     26    31   444 14185     3    85   245  4538     5     1]], shape=(1, 46), dtype=int32)


In [ ]:
print(tokenizer.decode(out[0], skip_special_tokens=True))

Google Translate est un service de traduction de machine neurale multilingue développé par Google pour traduire du texte, des documents et des sites Web d'une langue à une autre.


# Testing Original Model (No FineTuning)

In [ ]:
original_model = TFAutoModelForSeq2SeqLM.from_pretrained(model_id)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
input_text="Have you ever played soccer under the rain, with your friends? "
#input_text="Google Translate is a multilingual neural machine translation service developed by Google to translate text, documents and websites from one language into another."
input_text=prefix + input_text
tokenized=tokenizer([input_text], return_tensors='tf')
out = original_model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor(
[[    0    71  6071    18  3249  4315     3  8921   154    90 10749  2539
     50 26410     6   393  1507 15245    58     1]], shape=(1, 20), dtype=int32)


In [ ]:
print(tokenizer.decode(out[0], skip_special_tokens=True))

Avez-vous déjà joué le soccer sous la pluie, avec vos amis?
